https://gist.github.com/spro/ef26915065225df65c1187562eca7ec4

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
import numpy as np
import math, random
from tqdm import tqdm_notebook

from bokeh.plotting import figure, show, output_notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# Generating a noisy multi-sin wave 
class noisymultisin():
    def __init__(self):
        self.offset = 0
        
    def sine_2(self, X, signal_freq=60.):
        return (np.sin(2 * np.pi * (X) / signal_freq) + np.sin(4 * np.pi * (X) / signal_freq)) / 2.0

    def noisy(self, Y, noise_range=(-0.05, 0.05)):
        noise = np.random.uniform(noise_range[0], noise_range[1], size=Y.shape)
        return Y + noise

    def sample(self, sample_size):
        X = np.arange(sample_size)
        X += self.offset
        Y = self.noisy(self.sine_2(X + self.offset))
        self.offset += sample_size
        return X, Y

In [3]:
# generating a simple sin wave
class simplesin():
    def __init__(self):
        self.offset = 0
        
    def sin(self, X, signal_freq=60.):
        return np.sin(2 * np.pi * (X) / signal_freq)

    def sample(self, sample_size):
        X = np.arange(sample_size)
        X += self.offset
        Y = self.sin(X)
        self.offset += sample_size
        return X, Y

In [4]:
# Define the model

class SimpleLSTM(nn.Module):
    def __init__(self, hidden_size, dropout=0.05, num_layers=2):
        super(SimpleLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.dropout=dropout
        self.num_layers=num_layers
        self.batch_size = 1

        self.lstm = nn.LSTM(input_size=1, 
                            hidden_size=hidden_size, 
                            num_layers=self.num_layers, 
                            dropout=self.dropout,
                            batch_first=True)
        self.out = nn.Linear(hidden_size, 1)
        
        self.state = self.__default_state()

    def __default_state(self):
        return (torch.zeros(self.num_layers,
                            self.batch_size,
                            self.hidden_size, device=device),
                torch.zeros(self.num_layers,
                            self.batch_size,
                            self.hidden_size, device=device))

    def reset_state(self):
        self.state = self.__default_state()
    
    def printParams(self, index=None):
        if index is None:
            print("\nPrinting parameters")
            for parameter in self.parameters():
                print(parameter)
        else:
            for i, value in enumerate(self.parameters()):
                if i == index:
                    print(f"\nPrinting parameters at index {i}")
                    print(value)
                    break
    
    def adjust_learning_rate(optimizer, epoch):
        # Sets the learning rate to the initial LR decayed by 10 every 30 epochs
        lr = args.lr * (0.1 ** (epoch // 30))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    
    def evaluate(self, inputs, hidden):
        o, hidden = self.lstm(inputs, hidden)
        return(self.out(o.squeeze(1)), hidden)
        
    def forward(self, inputs, training=False):
        outputs = torch.zeros(inputs.size()[1], 1, 1).to(device)
        if training is True:
            # Training is done on all elements at once.
            # Leave states for predictions 
            self.reset_state()
            outputs, self.state = self.evaluate(inputs, self.state)
        else:
            input = torch.zeros(1,1,1).to(device)
            input[0,0,0] = inputs[0,0,0]
            for i in range(inputs.size()[1]):
                output, self.state = self.evaluate(input, self.state)
                outputs[i] = output
                output.unsqueeze_(0)
                input = output
            self.reset_state()
        return outputs

In [6]:
n_epochs=250
hidden_size=10
dropout=0.00
num_layers=2
learning_size=1400

model = SimpleLSTM(hidden_size=hidden_size, dropout=dropout, num_layers=num_layers)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

losses = np.zeros(n_epochs) # For plotting

wave = simplesin()
timesteps, _inputs = wave.sample(sample_size=learning_size)
inputs = torch.from_numpy(_inputs[:-1]).float().to(device)
targets = torch.from_numpy(_inputs[1:]).float().to(device)
inputs.unsqueeze_(0)
inputs.unsqueeze_(2)

for epoch in tqdm_notebook(range(n_epochs)):
 
    optimizer.zero_grad()

    outputs = model(inputs, training=True)
    outputs = torch.squeeze(outputs)
        
    loss = criterion(outputs.view(len(outputs)), targets)
    loss.backward()
    optimizer.step()
    losses[epoch] += loss.item()

In [7]:
output_notebook()

l = figure(plot_width=900, plot_height=400)
l.line(x=range(n_epochs), y=losses, line_width=2, line_color="blue")
l.xaxis.axis_label = "Epoch"
l.yaxis.axis_label = "Loss"
l.yaxis.major_label_orientation = "vertical"
show(l)

Loading BokehJS ...

In [8]:
output_notebook()

p = figure(plot_width=900, plot_height=400)
p.line(x=timesteps, y=_inputs, line_width=2, line_color="blue", legend="True")
p.line(x=timesteps[1:], y=outputs.tolist(), line_width=2, line_color="orange", legend="Prediction")
show(p)

Loading BokehJS ...


Try to do some prediction by taking more points in the sin wave


In [9]:
timesteps2, _inputs2 = wave.sample(sample_size=500)
inputs2 = torch.from_numpy(_inputs2[:-1]).float().to(device)
inputs2.unsqueeze_(0)
inputs2.unsqueeze_(2)
targets2 = torch.from_numpy(_inputs2[1:]).float()
outputs2 = model(inputs2)
outputs2 = torch.squeeze(outputs2)

In [11]:
output_notebook()

p = figure(plot_width=900, plot_height=400, title="Wave generation by model")

# add a line renderer
p.line(x=timesteps2, y=_inputs2, line_width=2, line_color="blue")
p.line(x=timesteps2[1:], y=outputs2.tolist(), line_width=2, line_color="orange")
show(p)

Loading BokehJS ...